In [ ]:
!pip install xlrd pandas
!pip install bnlearn

# Bayesian Networks

In this notebool, we will use bnlearn package to generate Bayesian Networks, based on structure learning and parameter learning. The generated Bayesian Networks will be used for inference. Using bnlearn package to learn a Bayesian network can be split into two problems:

- Structure learning: Given a set of data samples, estimate a DAG that captures the dependencies between the variables.

- Parameter learning: Given a set of data samples and a DAG that captures the dependencies between the variables, estimate the (conditional) probability distributions of the individual variables.

### Example: Building Bayesian Network and making inferences with Sprinkler data set

In [ ]:
import bnlearn as bn

# load the Sprinkler dataset
df = bn.import_example()

df

In [ ]:
# Structure Learning: obtaining the network structure

model = bn.structure_learning.fit(df)

G = bn.plot(model)

In [ ]:
# learn the parameters of the DAG using the df
model_update = bn.parameter_learning.fit(model, df, methodtype="bayes")

# Make plot
G = bn.plot(model_update)

In [ ]:
# performing inference
q_1 = bn.inference.fit(model_update, variables=['Wet_Grass'], evidence={'Cloudy':1,'Sprinkler':0, 'Rain':1})
q_2 = bn.inference.fit(model_update, variables=['Wet_Grass'], evidence={'Rain':1})

<div class='alert alert-block alert-success' style="font-weight:bolder">

### Task 1

Learn a Bayesian Network of the artificial smart grid data set. Construct the network make inference for the following:

- P(Outage_Duration=Less_than_1H | Time=Morning, Demand_Factor=Medium)
- P(Demand_Factor=High | Overload=Yes, Weather=Cold)
- P(Number_of_Customers=Low | Demand_Factor=High)


In [ ]:
import pandas as pd

file = "./smart_grid.xlsx"

df_sg = data = pd.read_excel(file)

print(df_sg.columns)
df_sg